In [ ]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [ ]:
# MNIST dataset parameters
num_classes = 10 # total classes (0-9 digits)
num_features = 784 # data features in our input (28*28 = 784)

# Training parameters
learning_rate = 0.001
training_steps = 3000
batch_size = 256
display_step = 100

# Neural Network parameters
n_hidden_1 = 128 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons

In [ ]:
# Prepare MNIST data.

# download the dataset

from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# convert to float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Flatten images to 1-D vector of 784 features (28*28)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Normalize image values from [0, 255] ---> [0, 1]
x_train, x_test = x_train / 255. , x_test / 255.

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Use tf.data API to shuffle and batch data

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:
# store layers weights and biases
# A random number generator to initialize weights

random_normal = tf.initializers.RandomNormal()

weights = {
    "h1": tf.Variable(random_normal([num_features, n_hidden_1])),
    "h2": tf.Variable(random_normal([n_hidden_1, n_hidden_2])),
    "out": tf.Variable(random_normal([n_hidden_2, num_classes]))
}

biases = {
    "b1": tf.Variable(random_normal([n_hidden_1])),
    "b2": tf.Variable(random_normal([n_hidden_2])),
    "out": tf.Variable(random_normal([num_classes]))
}

In [ ]:
print("Shape W_1: ", weights["h1"].shape)
print("Shape W_2: ", weights["h2"].shape)
print("Shape W_Out: ", weights["out"].shape)

print("Shape B_1: ", biases["b1"].shape)
print("Shape B_2: ", biases["b2"].shape)
print("Shape B_Out: ", biases["out"].shape)

Shape W_1:  (784, 128)
Shape W_2:  (128, 256)
Shape W_Out:  (256, 10)
Shape B_1:  (128,)
Shape B_2:  (256,)
Shape B_Out:  (10,)


In [ ]:
# Create model

def neural_net(x):
  # Hidden fully connected layer with 128 neuron.

  layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) # Z1

  # Apply sigmoid to layer_1 output for non-linearity

  layer_1 = tf.nn.sigmoid(layer_1)



  # Hidden fully connected layer with 256 neurons.

  layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) # Z2

  # apply sigmoid to layer_2 output

  layer_2 = tf.nn.sigmoid(layer_2) # A2



  # Output fully connected layer with a neuron for each class --> 10

  out_layer = tf.matmul(layer_2, weights['out']) + biases['out'] # Z3

  # Apply softmax to normalize the logits to a probability distributions

  return tf.nn.softmax(out_layer)

In [ ]:
# Cross-Entropy loss function
def cross_entropy(y_pred, y_true):
  # Encode label to a one hot vector
  y_true = tf.one_hot(y_true, depth = num_classes)

  # Clip predictions values to avoid log(0) error.

  y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

  # Compute cross-entropy loss

  return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metrics
def accuracy(y_pred, y_true):
  # Predicted class is the index of highest score in prediction vector( ie argmax)
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis = -1)

# Stochastic gradient descent optimizer
optimizer = tf.optimizers.SGD(learning_rate)

In [ ]:
# Optimizing process
def run_optimization(x, y):
  # Wrap computation inside a GradientTape for automatic differentiation

  with tf.GradientTape() as g:
    pred = neural_net(x)
    loss = cross_entropy(pred, y)

  # Variable to update ie trainable during back propagation

  trainable_variables = list(weights.values()) + list(biases.values())

  # Compute gradients
  gradients = g.gradient(loss, trainable_variables)

  # Update W and b following gradients
  optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
# Run training for the given number of steps
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
  # Run the optimization to update W and b values after ecah batch
  run_optimization(batch_x, batch_y)

  if step % display_step == 0:
    pred = neural_net(batch_x)
    loss = cross_entropy(pred, batch_y)
    acc = accuracy(pred, batch_y)

    print(f"step: {step}, loss: {loss}, accuracy: {acc}")

step: 100, loss: 564.4542846679688, accuracy: 0.27734375
step: 200, loss: 400.43096923828125, accuracy: 0.5625
step: 300, loss: 247.0797576904297, accuracy: 0.6953125
step: 400, loss: 189.78338623046875, accuracy: 0.80078125
step: 500, loss: 139.92044067382812, accuracy: 0.84765625
step: 600, loss: 141.63168334960938, accuracy: 0.828125
step: 700, loss: 89.343017578125, accuracy: 0.91796875
step: 800, loss: 122.954833984375, accuracy: 0.859375
step: 900, loss: 87.2774429321289, accuracy: 0.9140625
step: 1000, loss: 89.26688385009766, accuracy: 0.9140625
step: 1100, loss: 65.14659881591797, accuracy: 0.9296875
step: 1200, loss: 75.53355407714844, accuracy: 0.9296875
step: 1300, loss: 92.92730712890625, accuracy: 0.90234375
step: 1400, loss: 58.568214416503906, accuracy: 0.9453125
step: 1500, loss: 67.79147338867188, accuracy: 0.94140625
step: 1600, loss: 91.46562194824219, accuracy: 0.89453125
step: 1700, loss: 65.04832458496094, accuracy: 0.953125
step: 1800, loss: 79.53193664550781, a

In [ ]:
# Test model on validation set

pred = neural_net(x_test)

print(f"Test accurcay: {accuracy(pred, y_test)}")

Test accurcay: 0.9368000030517578


In [ ]:
# Visual prediction comparison

import matplotlib.pyplot as plt

In [18]:
# Predict 5 images from the validation set
n_images = 1000

test_images = x_test[:n_images]
# Shape = [5, 784]

predictions = neural_net(test_images)
# [5, 10]

# Display image and model prediction
for i in range(n_images):
  plt.imshow(np.reshape(test_images[i], [28, 28]), cmap = 'grey')
  plt.show()
  print(f"Model prediction {np.argmax(predictions. numpy()[i])}")

Output hidden; open in https://colab.research.google.com to view.